In [1]:
from google.colab import drive
drive.mount('/content/drive')
dataset_path = '/content/drive/My Drive/Datasets/Tic tac initial results.csv'


Mounted at /content/drive


In [40]:
df = pd.read_csv(dataset_path)
df.head()

,MOVE1,MOVE2,MOVE3,MOVE4,MOVE5,MOVE6,MOVE7,CLASS
0,0,8,1,3,?,?,?,loss
1,4,7,2,6,?,?,?,win
2,0,8,1,6,5,?,?,draw
3,4,7,2,3,?,?,?,draw
4,0,4,2,1,?,?,?,win


## 1. Data Preparation
You'll need to preprocess the dataset to create graph representations suitable for GNNs. Here's a step-by-step guide:

### 1. Load the Dataset:
Use pandas to load the dataset.
### 2. Graph Construction:
Create graphs where nodes represent board states and edges represent moves between states.
### 3. Feature Engineering:
Encode moves and outcomes as features.

## 2. Code Implementation
### 2.1 Load and Preprocess the Dataset

In [4]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.0 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder


# Encode class labels
label_encoder = LabelEncoder()
df['CLASS'] = label_encoder.fit_transform(df['CLASS'])

# Define a function to create a graph from the dataset row
def create_tic_tac_toe_graph(moves, outcome):
    board_size = 9  # Tic Tac Toe board is 3x3, so 9 cells
    node_features = np.zeros((board_size, 1))  # Initialize node features

    # Encode moves
    for i, move in enumerate(moves):
        if move != '?':
            node_features[int(move)] = 1  # Set the feature to 1 if move is present

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Create edges (moves can be seen as transitions from one board state to another)
    edge_index = []
    for i in range(len(moves) - 1):
        if moves[i] != '?' and moves[i + 1] != '?':
            edge_index.append([int(moves[i]), int(moves[i + 1])])

    # Handle the case where there are no edges
    if edge_index:
      edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    else:
      edge_index = torch.tensor([], dtype=torch.long).reshape(0, 2) # Create an empty tensor with the correct shape if no edges are present

    # Convert outcome to tensor
    outcome = torch.tensor(outcome, dtype=torch.long)

    return Data(x=node_features, edge_index=edge_index, y=outcome)

# Convert dataset to graph data
graph_data_list = []
for _, row in df.iterrows():
    moves = row[['MOVE1', 'MOVE2', 'MOVE3', 'MOVE4', 'MOVE5', 'MOVE6', 'MOVE7']].tolist()
    # Convert outcome to tensor
    outcome =  row['CLASS']
    graph_data = create_tic_tac_toe_graph(moves, outcome)
    graph_data_list.append(graph_data)


### 2.2 Define the GNN Model

In [6]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GNNModel(torch.nn.Module):
    def __init__(self):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16, 16)
        self.fc1 = torch.nn.Linear(16, 16)  # First fully connected layer
        self.fc2 = torch.nn.Linear(16, 9) # Output 9 scores for 9 moves Output layer)


    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Handle empty edge indices
        if edge_index.numel() == 0:
            x = torch.zeros_like(x)  # Set node features to zero if no edges
        else:
            x = F.relu(self.conv1(x, edge_index))
            x = F.relu(self.conv2(x, edge_index))

        x = torch.mean(x, dim=0, keepdim=True)  # Aggregate node features and keep dimensions

        # Ensure the shape is [1, 16] before feeding to fully connected layers
        if x.shape[1] != 16:
            x = torch.zeros(1, 16)  # If the shape isn't right, set it to zeros with the correct shape

        print("Reshaped x shape:", x.shape)

        x = F.relu(self.fc1(x))  # Pass through the first fully connected layer
        x = self.fc2(x)  # Pass through the second fully connected layer

        return x


latest

### 2.3 Training and Evaluation

In [7]:
from torch_geometric.loader import DataLoader

# Split data into train and test sets
train_size = int(0.8 * len(graph_data_list))
train_data = graph_data_list[:train_size]
test_data = graph_data_list[train_size:]

train_loader = DataLoader(train_data, batch_size=1, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

# Initialize model, optimizer, and loss function
model = GNNModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
def train():
    model.train()
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data)
        target = data.y.long()  # Ensure target is long and matches output dimensions
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        print(f'Train Loss: {loss.item()}')

# Evaluation loop
def evaluate():
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            out = model(data)
            _, predicted = torch.max(out, 1)  # Get the index of the max score
            total += data.y.size(0)
            correct += (predicted == data.y.view(-1)).sum().item()
        print(f'Accuracy: {correct / total * 100:.2f}%')


train()
evaluate()


Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.23468017578125
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.1711275577545166
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.111069440841675
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.038877010345459
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.032212972640991
Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.9235671758651733
Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.8540431261062622
Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.8073853254318237
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.5792136192321777
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.5706915855407715
Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.92050302028656
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.4998936653137207
Reshaped x shape: torch.Size([1, 16])
Train Loss: 1.8640024662017822
Reshaped x shape: torch.Size([1, 16])
Train Loss: 2.4681239128112793
Reshaped x shape: torch.Size([1, 16])
Tra

In [8]:
torch.save(model.state_dict(), 'gnn_model.pth')

print("Model saved as gnn_model.pth")

Model saved as gnn_model.pth


#Tic Tac Toe

In [35]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

# Define the GNN model
class GNNModel(torch.nn.Module):
    def __init__(self):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16, 16)
        self.fc1 = torch.nn.Linear(16, 16)
        self.fc2 = torch.nn.Linear(16, 9)  # Output 9 for each possible move

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = torch.mean(x, dim=0)  # Aggregate node features
        x = F.relu(self.fc1(x))   # Pass through first fully connected layer
        x = self.fc2(x)           # Pass through second fully connected layer
        return x

# Define the Tic Tac Toe game
class TicTacToe:
    def __init__(self):
        self.reset_game()
        self.model = self.load_model()  # Load your trained GNN model

    def reset_game(self):
        self.board = np.full((3, 3), '?')
        self.current_player = 'X'
        self.game_over = False
        self.winner = None
        print("Game has been reset.")

    def load_model(self):
        try:
            # Load the trained GNN model
            model = GNNModel()
            model.load_state_dict(torch.load('gnn_model.pth'))
            model.eval()
            print("Model loaded successfully.")
            return model
        except FileNotFoundError:
            print("Error: The model file 'gnn_model.pth' was not found.")
        except Exception as e:
            print(f"Error loading the model: {e}")

    def suggest_move(self, max_attempts=10):
        """
        Suggests a move for the AI player ('O').

        Parameters:
        - max_attempts (int): The maximum number of attempts to find a valid move.
        """
        if not self.game_over and self.current_player == 'O':  # AI's turn
            attempts = 0
            while attempts < max_attempts:
                try:
                    graph_data = self.create_graph_data(self.board)
                    with torch.no_grad():
                        output = self.model(graph_data)

                        # Convert model output to numpy array
                        output = output.squeeze().numpy()
                        print(f"Model output: {output}")

                        # Mask out invalid moves (already occupied positions)
                        valid_moves_mask = (self.board.flatten() == '?')
                        output[~valid_moves_mask] = -np.inf  # Assign -inf to already taken positions

                        # Find the index of the highest score among valid moves
                        move_index = np.argmax(output)
                        move = divmod(move_index, 3)

                        # Validate the suggested move
                        if self.is_valid_move(move):
                            self.make_move(*move)
                            print(f"AI makes move at position: {move}")
                            return  # Exit after making a valid move
                        else:
                            attempts += 1
                            print(f"Suggested move {move} is invalid or already taken.")
                except Exception as e:
                    print(f"Error during move suggestion: {e}")
                    attempts += 1

            # After exceeding max attempts, notify the user
            print("Max attempts reached. No valid moves found.")

    def is_valid_move(self, move):
        """
        Checks if a move is valid.

        Parameters:
        - move (tuple): The move to be checked, e.g., (row, col).

        Returns:
        - bool: True if the move is valid, False otherwise.
        """
        x, y = move
        return 0 <= x < 3 and 0 <= y < 3 and self.board[x, y] == '?'

    def create_graph_data(self, board):
        """
        Creates graph data from the Tic Tac Toe board.

        Parameters:
        - board (numpy.ndarray): The current state of the board.

        Returns:
        - Data: A PyTorch Geometric Data object representing the graph.
        """
        try:
            board = np.where(board == 'X', 1, np.where(board == 'O', -1, 0))
            node_features = torch.tensor(board, dtype=torch.float).view(-1, 1)

            edge_index = []
            for i in range(3):
                for j in range(3):
                    if i < 2:
                        edge_index.append([i * 3 + j, (i + 1) * 3 + j])
                    if j < 2:
                        edge_index.append([i * 3 + j, i * 3 + (j + 1)])

            edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
            print("Graph data created successfully.")
            return Data(x=node_features, edge_index=edge_index)
        except Exception as e:
            print(f"Error creating graph data: {e}")
            raise

    def make_move(self, x, y):
        if not self.game_over and self.board[x, y] == '?':
            self.board[x, y] = self.current_player
            print(f"Player {self.current_player} makes a move at ({x}, {y})")
            if self.check_winner():
                self.game_over = True
                self.winner = self.current_player
                print(f"Player {self.winner} wins!")
            elif np.all(self.board != '?'):
                self.game_over = True
                self.winner = 'Draw'
                print("The game is a draw!")
            else:
                self.current_player = 'O' if self.current_player == 'X' else 'X'
                self.suggest_move()
            self.update_ui()

    def check_winner(self):
        for i in range(3):
            if np.all(self.board[i, :] == self.current_player) or np.all(self.board[:, i] == self.current_player):
                return True
        if self.board[0, 0] == self.current_player and self.board[1, 1] == self.current_player and self.board[2, 2] == self.current_player:
            return True
        if self.board[0, 2] == self.current_player and self.board[1, 1] == self.current_player and self.board[2, 0] == self.current_player:
            return True
        return False

    def update_ui(self):
        for i in range(3):
            for j in range(3):
                button = self.buttons[i][j]
                button.description = self.board[i, j]
                button.disabled = self.board[i, j] != '?'
        if self.game_over:
            if self.winner == 'Draw':
                result_text = 'The game is a draw!'
            else:
                result_text = f'Player {self.winner} wins!'
            self.result_label.value = result_text
            print("UI updated.")

    def create_ui(self):
        self.buttons = [[None for _ in range(3)] for _ in range(3)]
        button_grid = widgets.GridBox(
            children=[widgets.Button(description='?', layout=widgets.Layout(width='100px', height='100px')) for _ in range(9)],
            layout=widgets.Layout(grid_template_columns='repeat(3, 100px)')
        )
        self.result_label = widgets.Label(value='')
        self.ui = widgets.VBox([button_grid, self.result_label])
        for i in range(3):
            for j in range(3):
                button = button_grid.children[i * 3 + j]
                button.on_click(lambda btn, x=i, y=j: self.make_move(x, y))
                self.buttons[i][j] = button
        self.update_ui()  # Update UI after creating it
        print("UI created.")
        return self.ui

# Create and display the game UI
game = TicTacToe()
display(game.create_ui())


Game has been reset.
Model loaded successfully.
UI created.


Player X makes a move at (1, 0)
Graph data created successfully.
Model output: [-2.6283996  1.024118   1.8419336 -8.30699   -9.278661  -8.708213
 -9.076064  -8.809577  -8.716147 ]
Player O makes a move at (0, 2)
AI makes move at position: (0, 2)
Player X makes a move at (2, 2)
Graph data created successfully.
Model output: [-2.5345702   0.99833995  1.8001095  -8.0176115  -8.936362   -8.396335
 -8.739559   -8.495468   -8.398293  ]
Player O makes a move at (0, 1)
AI makes move at position: (0, 1)
Player X makes a move at (0, 0)
Graph data created successfully.
Model output: [  2.7851863    0.834722    -0.06506208 -10.985558    -8.84322
  -9.969808    -9.4609165   -9.981172    -9.498582  ]
Player O makes a move at (1, 1)
AI makes move at position: (1, 1)
Player X makes a move at (1, 2)
Graph data created successfully.
Model output: [  3.0055168    0.87803835  -0.0995039  -11.69428     -9.402637
 -10.591571   -10.091726   -10.635295   -10.126087  ]
Player O makes a move at (2, 0)
Player O 

In [41]:
#link = 'https://drive.google.com/file/d/1eMCrvxXSj-NdyPNLMgiMOu6-8XwXY58H/view?usp=drive_link'
# Replace 'FILE_ID' with your actual file ID from the Google Drive link
file_id = 'd/1eMCrvxXSj-NdyPNLMgiMOu6-8XwXY58H'
# Create the download URL
download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

